In [4]:
from pathlib import Path

from numba import njit, f8
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, minimize

from density_estimation.dist import (
    Normal, 
    StudentT,
    Laplace,
    CondSNorm, 
    CondST,
    CondSLap,
    CondJsu
)
from density_estimation.models.simple import ArGarch
from density_estimation.common import sumjit, get_data, kurtosis
from density_estimation.har import har_fitness, calc_har

In [4]:
def eval_garch(model, log_returns):
    forecast = model(log_returns)
    sigma = np.sqrt(forecast[:, 1])
    lprob = model.error_dist.llh(forecast[:, 0] / sigma) - sumjit(np.log(sigma))
    crp_scores = model.error_dist.crps(log_returns, forecast[:, 0], sigma)
    return -lprob / len(data), np.mean(crp_scores)

def eval_har(params, dist_cls, data):
    log_returns, rv_dmw = data[:,0], data[:, 1:]
    sigma = forecast_rv_d(rv_dmw, params[:4])
    error_dist = dist_cls(*params[4:])
    lprob = error_dist.llh(log_returns / sigma) - sumjit(np.log(sigma))
    crp_scores = error_dist.crps(log_returns, np.zeros(log_returns.size), sigma)
    return -lprob / len(data), np.mean(crp_scores)

In [5]:
distributions = {
    'normal':{'cls': Normal, 'shape_guess': []}, 
    't':{'cls': StudentT, 'shape_guess': []},
    'laplace':{'cls': Laplace, 'shape_guess': []},
    'skewnorm':{'cls': CondSNorm, 'shape_guess':[1.0]}, 
    'skewt':{'cls': CondST, 'shape_guess':[]},
    'skewlap':{'cls': CondSLap, 'shape_guess':[1.0]},
    'jsu':{'cls': CondJsu, 'shape_guess':[0.0, 1.0]}
}

In [6]:
# Calculating the CRPS is currently a huge bottleneck, removing CRPS from eval 
# function reduces total time from ~20 miutes to ~5 seconds.

results = []
for csv_path in Path('data/TAQ').iterdir():
    company = csv_path.stem.split('_')[0]
    data = get_data(csv_path)
    sample_T = data.shape[0] // 10 * 8
    sample, oos = data[: sample_T], data[sample_T: ]

    # calc values for initial guesses
    lr_mean, drv_mean = sample[:, :2].mean(axis=0)
    lr_var = sample[:, 0].var(ddof=1)
    lr_kurt = kurtosis(sample[:, 0], lr_mean, lr_var)
    distributions['t']['shape_guess'] = [lr_kurt]
    distributions['skewt']['shape_guess'] = [1.0, lr_kurt]
    
    for distribution, ddict in distributions.items():
        dist = ddict['cls']

        initial = np.array([lr_mean, 0.0, lr_var, 0.05, 0.9, *ddict['shape_guess']], dtype=np.float64)
        argarch_model = ArGarch(dist)
        res = argarch_model.fit(sample[:, 0], initial, display=False, maxiter=300)
        if res.success:
            lsr, crps = eval_garch(argarch_model, oos[:, 0])
            results.append([company, 'garch', dist.name, 5 + dist.n_params, res.fun, lsr, crps])
        else:
            print(f'GARCH {dist.name} for {company} failed with {res.message}')
            results.append([company, 'garch', dist.name, 5 + dist.n_params, np.nan, np.nan, np.nan])

        initial = np.array([drv_mean, 0.3, 0.3, 0.3, *ddict['shape_guess']])
        har_bounds = Bounds(
            lb=np.array([1e-8, 0., 0., 0., *dist.bounds.lb]),
            ub=np.array([np.inf, np.inf, np.inf, np.inf, *dist.bounds.ub])
        )
        res = minimize(
            har_fitness, initial,
            args=(sample, dist),
            bounds=har_bounds,
            constraints=[{'type': 'ineq', 'fun': lambda x: sum(x[1:4])}],
            method='SLSQP',
            options={"disp": False}
        )
        if res.success:
            lsr, crps = eval_har(res.x, dist, oos)
            results.append([company, 'har', dist.name, 4 + dist.n_params, res.fun, lsr, crps])
        else:
            print(f'HAR {dist.name} for {company} failed with {res.message}')
            results.append([company, 'har', dist.name, 4 + dist.n_params, np.nan, np.nan, np.nan])

/home/wswall/ParDen/models/simple.py:80: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(fit_data[:, 1])


GARCH laplace for jpm failed with Singular matrix E in LSQ subproblem
GARCH skewlap for jpm failed with Singular matrix E in LSQ subproblem


/home/wswall/ParDen/models/simple.py:80: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(fit_data[:, 1])


GARCH skewt for ge failed with Singular matrix E in LSQ subproblem


/home/wswall/ParDen/dist.py:229: RuntimeWarning: divide by zero encountered in log
  Z = self.params['skew'] + self.params['tailweight'] * np.log(z + np.sqrt(z ** 2 + 1))
/home/wswall/ParDen/dist.py:242: RuntimeWarning: divide by zero encountered in log
  return sumjit(np.log(self.pdf(z_std)))


CPU times: user 16min 22s, sys: 1min 7s, total: 17min 29s
Wall time: 17min 29s


In [ ]:
cols = ['company', 'model_type', 'distribution', 'k', 'llh', 'lsr', 'crps']
score_df = pd.DataFrame(results, columns=cols)

score_df['llh'] = -score_df.llh * 1e5
score_df['aic'] = (2 * score_df.k - 2 * score_df.llh) / sample_T
score_df['bic'] = (np.log(sample_T) * score_df.k - 2 * score_df.llh) / sample_T
score_df['crps'] = score_df.crps * 1e3
score_df.rename(columns={'crps': 'crps_x1e3'}, inplace=True)
float_cols = score_df.select_dtypes(include=float).columns
score_df[float_cols] = score_df[float_cols].round(3)

write_order = ['company', 'model_type', 'distribution', 'k', 'llh', 'aic', 'bic', 'lsr', 'crps_x1e3']
score_df[write_order].to_csv('results/eval_results.csv', index=False)

In [40]:
data = get_data('data/ibm_300_cts.csv')

19.3 ms ± 3.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
sample = data[:1367]
daily_rv = sample[:, 1]

In [54]:
res = minimize(
    har_fitness, 
    np.array([daily_rv.mean(), 0.3, 0.3, 0.3]), 
    args=(sample, Normal),
    bounds=Bounds(
        lb=np.array([1e-8, 0., 0., 0.]),
        ub=np.array([np.inf, np.inf, np.inf, np.inf])
    ),
    constraints=[{'type': 'ineq', 'fun': lambda x: sum(x[1:4])}],
    method='SLSQP',
    options={"disp": True}
)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.04135557330124296
            Iterations: 11
            Function evaluations: 57
            Gradient evaluations: 11


In [55]:
res.x

array([1.00000000e-08, 3.36964675e-01, 3.45623285e-01, 3.38158938e-01])

In [ ]:
ArGarch(Normal).fit(
    lr, 
    np.array([lr.mean(), 0.0, lr.var(), 0.05, 0.9]), 
    display=False
)

In [41]:
data = get_data('data/ibm_300_cts.csv')
sample_T = data.shape[0] // 10 * 8
sample, oos = data[: sample_T], data[sample_T: ]
lr = data[:,0]
lr_mean, drv_mean = sample[:, :2].mean(axis=0)
lr_var = sample[:, 0].var(ddof=1)
lr_kurt = kurtosis(sample[:, 0], lr_mean, lr_var)
initial = np.array([data[:, 1].mean(), 0.3, 0.3, 0.3, 1.0, lr_kurt])
dist = CondST
bounds = Bounds(
    lb=np.array([1e-8, 0., 0., 0., *dist.bounds.lb]),
    ub=np.array([np.inf, np.inf, np.inf, np.inf, *dist.bounds.ub])
    )
constraints = [{'type': 'ineq', 'fun': lambda x: sum(x[1:4])}]